In [1]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession

In [2]:
conf = SparkConf().setAppName("Lab1_Nepryakhin").setMaster('yarn')

In [3]:
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

In [4]:
tripData = spark.read\
.option("header", True)\
.option("inferSchema", True)\
.option("timestampFormat", 'M/d/y H:m')\
.csv("trips.txt")

In [5]:
stationData = spark.read\
.option("header", True)\
.option("inferSchema", True)\
.option("timestampFormat", 'M/d/y')\
.csv("stations.txt")

In [6]:
stationData.createOrReplaceTempView("stations")
tripData.createOrReplaceTempView("trips")

#### Найти количество велосипедов в системе

In [8]:
endTrips = spark.sql("""
SELECT COUNT(DISTINCT bike_id) AS Kol_vo_velosipedov_v_sisteme
FROM trips
""")
endTrips.show()


+----------------------------+
|Kol_vo_velosipedov_v_sisteme|
+----------------------------+
|                         700|
+----------------------------+



#### Найти велосипед с максимальным временем пробега

In [10]:
endTrips = spark.sql("""
SELECT bike_id, SUM(duration) AS duration_sum
FROM trips
    GROUP BY bike_id
    ORDER BY duration_sum DESC
    LIMIT 1 
""")
endTrips.show()

+-------+------------+
|bike_id|duration_sum|
+-------+------------+
|    535|    36229902|
+-------+------------+



#### Найти наибольшее геодезическое расстояние между станциями

In [12]:
endTrips = spark.sql("""
SELECT from_city, to_city, max(distance_in_km)  as max_dist
    FROM(
    SELECT a.name AS from_city, b.name AS to_city, 
       111.111 *
        DEGREES(ACOS(LEAST(1.0, COS(RADIANS(a.lat))
             * COS(RADIANS(b.lat))
             * COS(RADIANS(a.long - b.long))
             + SIN(RADIANS(a.lat))
             * SIN(RADIANS(b.lat))))) AS distance_in_km
      FROM stations AS a
      JOIN stations AS b ON a.id <> b.id ) AS dist
      GROUP BY from_city, to_city
      ORDER BY max_dist DESC
      limit 1

""")
endTrips.show()


+--------------------+--------------------+-----------------+
|           from_city|             to_city|         max_dist|
+--------------------+--------------------+-----------------+
|Embarcadero at Sa...|SJSU - San Salvad...|69.86810174337214|
+--------------------+--------------------+-----------------+



#### Найти путь велосипеда с максимальным временем пробега через станции.

In [14]:
endTrips = spark.sql("""
SELECT id, start_station_name, end_station_name, duration
FROM trips
    WHERE bike_id = 535
    ORDER BY start_date
""")
endTrips.show()

+-----+--------------------+--------------------+--------+
|   id|  start_station_name|    end_station_name|duration|
+-----+--------------------+--------------------+--------+
| 4966|     Post at Kearney|San Francisco Cal...|    1245|
| 5067|San Francisco Cal...|San Francisco Cal...|     423|
| 5179|San Francisco Cal...|   Market at Sansome|     842|
| 5199|   Market at Sansome|   2nd at South Park|     498|
| 7806|     2nd at Townsend|    Davis at Jackson|    1671|
|11422|San Francisco Cit...|Civic Center BART...|     260|
|12245|Civic Center BART...|     Post at Kearney|    1192|
|12485|     Post at Kearney|Embarcadero at Sa...|    1248|
|12558|Embarcadero at Sa...|Washington at Kea...|    1272|
|13107|Washington at Kea...|   Market at Sansome|     398|
|13423|   Market at Sansome|   Market at Sansome|   12476|
|14380|   Market at Sansome|       2nd at Folsom|     582|
|14581|       2nd at Folsom|     2nd at Townsend|     499|
|15231|Temporary Transba...|     2nd at Townsend|     46

#### Найти пользователей потративших на поездки более 3 часов.

In [16]:
endTrips = spark.sql("""
SELECT 
    zip_code, 
    SUM (duration) AS sum_duration
FROM trips
    GROUP BY zip_code
    HAVING sum_duration > 180
    ORDER BY sum_duration
""")
endTrips.show()


+--------+------------+
|zip_code|sum_duration|
+--------+------------+
|   87031|         188|
|   33637|         190|
|   93740|         196|
|   64610|         208|
|   28005|         232|
|   60010|         234|
|   34531|         238|
|   80106|         259|
|   11566|         264|
|   50225|         270|
|   91064|         278|
|   39057|         300|
|    9431|         317|
|    9999|         332|
|   60624|         332|
|   94950|         341|
|    1651|         342|
|   21756|         343|
|   77253|         349|
|   28879|         364|
+--------+------------+
only showing top 20 rows

